In [73]:
import torch

from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import fpsample

In [74]:
mesh_path = "data/armadillo_curvs.ply"
mesh = trimesh.load(mesh_path)
vertices = np.array(mesh.vertices)
sampled_indices = fpsample.fps_sampling(vertices, 10, start_idx=0) # the start index is for reproducibility
print(sampled_indices)
# dataset_full_sample = create_dataset_from_mesh(mesh_path=mesh_path, sampled_index=0, show_sample=True, ball_radius=0.05, seed=0, down_sampling_ratio=1.0)
# print(dataset_full_sample["train_input"].shape)

[     0    951   8603 104519   9116 115547  11546  42974 114773 169557]


In [75]:
grid_range = 1.0
ball_radius = 0.06 # for bunny worked good with 0.09 around 250 points neighbors
lr = 1.0
lamb = 100.0

signatures_full_sampling = torch.zeros((len(sampled_indices), 6), dtype=torch.float32)
signatures_down_sampled = torch.zeros((len(sampled_indices), 6), dtype=torch.float32)
for i,sampled_index in enumerate(sampled_indices):
    dataset_full_sample = create_dataset_from_mesh(mesh_path=mesh_path, sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=1.0)
    print("dataset full size:" , len(dataset_full_sample["train_input"]))
    model_full_sampled = KAN(width=[2, 5, 1], grid=1, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range, grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=20, lr=lr, lamb=lamb, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
    model_full_sampled = model_full_sampled.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0, 1], [0]])
    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
     # manual mode
    # if nan continue

    # fix all
    try:
        model_full_sampled.fix_symbolic(0, 0, 0, 'x^4')
        model_full_sampled.fix_symbolic(0, 1, 0, 'x^3')
        model_full_sampled.fix_symbolic(1, 0, 0, 'x^2')
    except:
        print("nan encountered")
        continue


    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, lamb_coef=0.10, lamb_coefdiff=0.10, loss_fn=gaussian_weighted_mse)

    print(model_full_sampled.symbolic_formula()[0][0])
    x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

    signature_full = calculate_signature(model_full_sampled, x) # only calc sig in the origin


    # down sampling

    dataset_down_sampled = create_dataset_from_mesh(mesh_path=mesh_path, sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=0.8)
    print("dataset down sampled size:" , len(dataset_down_sampled["train_input"]))
    model_down_sampled = KAN(width=[2, 5, 1], grid=1, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range, grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=20, lr=lr, lamb=lamb, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
    model_down_sampled = model_full_sampled.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0, 1], [0]])
    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
     # manual mode

    # fix all
    try:
        model_down_sampled.fix_symbolic(0, 0, 0, 'x^4')
        model_down_sampled.fix_symbolic(0, 1, 0, 'x^3')
        model_down_sampled.fix_symbolic(1, 0, 0, 'x^2')
    except:
        print("nan encountered")
        continue

    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, lamb_coef=0.10, lamb_coefdiff=0.10, loss_fn=gaussian_weighted_mse)

    print(model_down_sampled.symbolic_formula()[0][0])
    x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

    signature_down_sampled = calculate_signature(model_down_sampled, x) # only calc sig in the origin
    # check if signature_full or signature_down_sample is nan
    if torch.isnan(signature_full).any(dim=1) or torch.isnan(signature_down_sampled).any(dim=1):
        print("nan encountered")
        continue

    print("sampled_index", i, "signature ", signature_full)
    signatures_full_sampling[i,:] = signature_full
    print("sampled_index", i, "signature ", signature_down_sampled)
    signatures_down_sampled[i,:] = signature_down_sampled



Number of vertices: 172974
dataset full size: 334


train loss: 1.70e-02 | test loss: 1.79e-02 | reg: 3.10e+00 : 100%|██| 20/20 [00:02<00:00,  7.50it/s]
train loss: 2.08e-03 | test loss: 3.06e-03 | reg: 4.14e+00 : 100%|██| 50/50 [00:02<00:00, 18.36it/s]


r2 is 0.9870117902755737
r2 is 0.9114292860031128
r2 is 0.9171762466430664


train loss: 4.56e-03 | test loss: 5.70e-03 | reg: 1.05e+01 : 100%|██| 50/50 [00:00<00:00, 54.00it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.acts_scale_std.append(torch.std(postacts, dim=0))


265010.15*((0.08 - x_1)**4 + 0.14*(-x_2 - 0.07)**3)**2 - 0.01
Number of vertices: 172974
dataset down sampled size: 268


...

47026.73*(0.03*(0.07 - x_2)**3 + (0.1 - x_1)**4)**2 - 0.02
Number of vertices: 172974
dataset down sampled size: 268


train loss: 6.73e-03 | test loss: 6.72e-03 | reg: 3.19e+00 : 100%|██| 20/20 [00:02<00:00,  9.17it/s]
train loss: 2.56e-03 | test loss: 3.05e-03 | reg: 5.80e+00 : 100%|██| 50/50 [00:00<00:00, 68.79it/s]


Best value at boundary.
r2 is 0.9999982118606567
r2 is 0.999930739402771
Best value at boundary.
r2 is 0.9967049360275269


train loss: 2.56e-03 | test loss: 3.05e-03 | reg: 5.86e+00 : 100%|██| 50/50 [00:00<00:00, 68.14it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.acts_scale_std.append(torch.std(postacts, dim=0))


52001.56*(0.03*(0.07 - x_2)**3 + (0.1 - x_1)**4)**2 - 0.02
sampled_index 9 signature  tensor([[  3.2499,   4.1720,   2.2970,  -7.1953,  45.0965, -41.2163]],
       grad_fn=<StackBackward0>)
sampled_index 9 signature  tensor([[  3.2499,   4.2133,   1.9376, -11.6430,  44.0543, -50.2017]],
       grad_fn=<StackBackward0>)


In [76]:
indices_full_thrown = torch.isnan(signatures_full_sampling).any(dim=1)
indices_full_thrown =torch.concatenate([indices_full_thrown, signatures_full_sampling.norm(dim=1) == 0])
indices_down_sampled_thrown = torch.isnan(signatures_down_sampled).any(dim=1)
indices_down_sampled_thrown = torch.concatenate([indices_down_sampled_thrown, signatures_down_sampled.norm(dim=1) == 0])

# throw 0's
signatures_full_sampling = signatures_full_sampling[signatures_full_sampling.norm(dim=1) != 0]
signatures_down_sampled = signatures_down_sampled[signatures_down_sampled.norm(dim=1) != 0]
# throw nan's
signatures_full_sampling = signatures_full_sampling[~torch.isnan(signatures_full_sampling).any(dim=1)]
signatures_down_sampled = signatures_down_sampled[~torch.isnan(signatures_down_sampled).any(dim=1)]


In [77]:
# indice_thrown indices, not logical
# indices_full_thrown = torch.concatenate(indices_full_thrown)
print(indices_full_thrown)
print(torch.arange(len(indices_full_thrown))[indices_full_thrown])

print(indices_down_sampled_thrown.shape)
print(torch.arange(len(indices_down_sampled_thrown))[indices_down_sampled_thrown])

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])
tensor([], dtype=torch.int64)
torch.Size([20])
tensor([], dtype=torch.int64)


In [78]:
# normalize each entry to 1 in each of the 6 columns of the signature
signatures_full_sampling_normalized = signatures_full_sampling / torch.norm(signatures_full_sampling, dim=0)
signatures_down_sampled_normalized = signatures_down_sampled / torch.norm(signatures_down_sampled, dim=0)
print(signatures_full_sampling_normalized)
print(signatures_down_sampled_normalized)

tensor([[-3.5560e-02, -8.5463e-01, -3.4187e-01,  2.4566e-03, -9.6156e-01,
         -2.1712e-01],
        [-6.5545e-04, -4.5480e-01, -2.2331e-02,  7.7412e-04, -2.2929e-02,
          1.4290e-02],
        [ 9.4696e-01, -1.4025e-02, -9.0909e-01, -4.6661e-01, -2.7711e-03,
         -9.4217e-01],
        [ 7.2696e-02,  9.1817e-02,  7.0359e-03,  1.0883e-01,  7.1894e-03,
          2.2651e-01],
        [ 6.2572e-03,  4.2141e-03,  2.0474e-02, -8.3808e-04,  1.6890e-03,
         -6.5010e-03],
        [ 3.0796e-01,  1.2018e-01,  2.6004e-02, -8.7773e-01,  2.4899e-01,
          5.8429e-02],
        [-4.1333e-03, -3.2609e-02, -2.0649e-01,  2.5057e-03, -7.8359e-02,
         -1.4965e-02],
        [-1.2539e-02, -1.7419e-01,  1.0325e-01,  3.3904e-03,  7.5065e-02,
         -9.7978e-02],
        [-2.2819e-02, -2.2201e-02, -4.1430e-02, -5.6585e-04,  2.5616e-02,
         -1.3044e-03],
        [ 3.3829e-02,  8.8180e-02,  4.8332e-03, -1.0329e-03,  1.9742e-02,
         -1.9557e-02]], grad_fn=<DivBackward0>)
tenso

In [79]:
# measure correspondence according distance between signatures
# before normalized each entry to 1

distances = torch.cdist(signatures_full_sampling_normalized, signatures_down_sampled_normalized, p=2)
print(distances)


tensor([[3.0736e-02, 1.1099e+00, 1.8050e+00, 1.5873e+00, 1.3573e+00, 1.9936e+00,
         1.2622e+00, 1.3202e+00, 1.3431e+00, 1.4136e+00],
        [1.1167e+00, 4.1809e-02, 1.5955e+00, 7.4250e-01, 4.6204e-01, 1.2768e+00,
         4.3194e-01, 3.4990e-01, 4.3721e-01, 5.4145e-01],
        [1.9111e+00, 1.7248e+00, 3.7012e-01, 1.9275e+00, 1.6858e+00, 1.8141e+00,
         1.6085e+00, 1.6975e+00, 1.6711e+00, 1.6556e+00],
        [1.5002e+00, 5.6447e-01, 1.6452e+00, 1.7074e-01, 2.8012e-01, 1.1657e+00,
         3.4421e-01, 4.2417e-01, 2.9820e-01, 2.7587e-01],
        [1.3809e+00, 4.2082e-01, 1.5408e+00, 4.4188e-01, 4.6625e-04, 1.1472e+00,
         1.6733e-01, 1.9122e-01, 7.9472e-02, 8.6953e-02],
        [1.8957e+00, 1.1089e+00, 1.6315e+00, 1.0982e+00, 9.6897e-01, 3.4986e-01,
         1.0089e+00, 1.0046e+00, 9.7844e-01, 9.5131e-01],
        [1.2528e+00, 4.2686e-01, 1.4188e+00, 5.2462e-01, 2.4365e-01, 1.2001e+00,
         8.2183e-02, 3.4827e-01, 1.9459e-01, 2.6105e-01],
        [1.3453e+00, 3.0900

In [80]:
# take minimum distance for each row and assign index which is the closest for each surface
min_distances, min_indices = torch.min(distances, dim=1)
print(min_distances)
print(min_indices)

tensor([0.0307, 0.0418, 0.3701, 0.1707, 0.0005, 0.3499, 0.0822, 0.0394, 0.0028,
        0.0073], grad_fn=<MinBackward0>)
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [81]:
signatures_full_sampling_normalized

tensor([[-3.5560e-02, -8.5463e-01, -3.4187e-01,  2.4566e-03, -9.6156e-01,
         -2.1712e-01],
        [-6.5545e-04, -4.5480e-01, -2.2331e-02,  7.7412e-04, -2.2929e-02,
          1.4290e-02],
        [ 9.4696e-01, -1.4025e-02, -9.0909e-01, -4.6661e-01, -2.7711e-03,
         -9.4217e-01],
        [ 7.2696e-02,  9.1817e-02,  7.0359e-03,  1.0883e-01,  7.1894e-03,
          2.2651e-01],
        [ 6.2572e-03,  4.2141e-03,  2.0474e-02, -8.3808e-04,  1.6890e-03,
         -6.5010e-03],
        [ 3.0796e-01,  1.2018e-01,  2.6004e-02, -8.7773e-01,  2.4899e-01,
          5.8429e-02],
        [-4.1333e-03, -3.2609e-02, -2.0649e-01,  2.5057e-03, -7.8359e-02,
         -1.4965e-02],
        [-1.2539e-02, -1.7419e-01,  1.0325e-01,  3.3904e-03,  7.5065e-02,
         -9.7978e-02],
        [-2.2819e-02, -2.2201e-02, -4.1430e-02, -5.6585e-04,  2.5616e-02,
         -1.3044e-03],
        [ 3.3829e-02,  8.8180e-02,  4.8332e-03, -1.0329e-03,  1.9742e-02,
         -1.9557e-02]], grad_fn=<DivBackward0>)

In [82]:
signatures_down_sampled_normalized

tensor([[-3.3742e-02, -8.6945e-01, -3.5843e-01,  2.6541e-03, -9.7073e-01,
         -2.3619e-01],
        [-1.7788e-04, -4.1301e-01, -2.3237e-02,  8.4123e-04, -2.3189e-02,
          1.4447e-02],
        [ 9.2262e-01, -4.4633e-03, -9.1812e-01, -1.2380e-01, -7.8625e-04,
         -8.0542e-01],
        [ 8.5033e-02,  1.7545e-01,  1.7603e-02,  1.2147e-01,  1.3785e-02,
          3.7378e-01],
        [ 6.3710e-03,  4.1057e-03,  2.0343e-02, -8.1287e-04,  1.4275e-03,
         -6.1745e-03],
        [ 3.7226e-01,  1.1051e-01,  2.1218e-02, -9.8483e-01,  2.2518e-01,
          3.8418e-01],
        [-9.2280e-03, -3.9443e-02, -1.3364e-01,  2.8480e-03, -4.1746e-02,
         -2.0719e-02],
        [-9.8330e-03, -1.4721e-01,  8.3979e-02,  3.8344e-03,  5.9523e-02,
         -8.3802e-02],
        [-2.3156e-02, -2.1721e-02, -4.4135e-02, -6.7394e-04,  2.5699e-02,
         -1.2911e-03],
        [ 3.2789e-02,  8.2176e-02,  4.0872e-03, -1.8363e-03,  1.7499e-02,
         -2.2609e-02]], grad_fn=<DivBackward0>)

In [83]:
torch.norm(signatures_full_sampling_normalized[0] - signatures_down_sampled_normalized[1])

tensor(1.1099, grad_fn=<LinalgVectorNormBackward0>)

In [84]:
# sum all the min_indices whice equal their index, for example if 0 is in 0 index, add 1 to the counter
torch.sum(min_indices == torch.arange(len(min_indices)))

tensor(10)

In [85]:
# test of distances only using the first entry of the signature, then the first and second, then the first, second and third and so on til the fifth
for i in range(6):
    distances = torch.cdist(signatures_full_sampling_normalized[:, :i+1], signatures_down_sampled_normalized[:, :i+1], p=2)
    min_distances, min_indices = torch.min(distances, dim=1)
    print("i", i, "min_distances", min_distances, "min_indices", min_indices)
    print("i", i, "sum", torch.sum(min_indices == torch.arange(len(min_indices))))

i 0 min_distances tensor([0.0018, 0.0005, 0.0243, 0.0123, 0.0001, 0.0643, 0.0040, 0.0027, 0.0003,
        0.0010], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 3, 4, 5, 1, 7, 8, 9])
i 0 sum tensor(9)
i 1 min_distances tensor([0.0149, 0.0418, 0.0262, 0.0411, 0.0002, 0.0650, 0.0085, 0.0271, 0.0006,
        0.0061], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 9, 4, 5, 6, 7, 8, 9])
i 1 sum tensor(9)
i 2 min_distances tensor([0.0223, 0.0418, 0.0277, 0.0412, 0.0002, 0.0652, 0.0733, 0.0333, 0.0028,
        0.0061], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 9, 4, 5, 6, 7, 8, 9])
i 2 sum tensor(9)
i 3 min_distances tensor([2.2292e-02, 4.1808e-02, 3.4392e-01, 8.6129e-02, 2.0600e-04, 1.2538e-01,
        7.3351e-02, 3.3270e-02, 2.7702e-03, 6.1912e-03],
       grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
i 3 sum tensor(10)
i 4 min_distances tensor([2.4104e-02, 4.1809e-02, 3.4393e-01, 8.6381e-02, 3.3290e-04, 1.2763e-01,
        8.1981e-02, 3.

In [86]:
# i want to check per entry which one is the closest and then do a major vote
# for example surface i first entry is closest to surface j and second is closest to k and so on then i new_sig = [j,k,..,..,...] in the end we majority vote which closest, if j appear the most in i new sig then i is closest to j
for i in range(6):
    distances = torch.cdist(signatures_full_sampling_normalized[:, i:i+1], signatures_down_sampled_normalized[:, i:i+1], p=2)
    min_distances, min_indices = torch.min(distances, dim=1)
    print("i", i, "min_distances", min_distances, "min_indices", min_indices)
    print("i", i, "sum", torch.sum(min_indices == torch.arange(len(min_indices))))


i 0 min_distances tensor([0.0018, 0.0005, 0.0243, 0.0123, 0.0001, 0.0643, 0.0040, 0.0027, 0.0003,
        0.0010], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 3, 4, 5, 1, 7, 8, 9])
i 0 sum tensor(9)
i 1 min_distances tensor([0.0148, 0.0418, 0.0077, 0.0096, 0.0001, 0.0097, 0.0068, 0.0270, 0.0005,
        0.0060], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 8, 9, 4, 5, 6, 7, 8, 9])
i 1 sum tensor(8)
i 2 min_distances tensor([0.0166, 0.0009, 0.0090, 0.0029, 0.0001, 0.0048, 0.0729, 0.0193, 0.0027,
        0.0007], grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 9, 4, 5, 6, 7, 8, 9])
i 2 sum tensor(9)
i 3 min_distances tensor([1.9743e-04, 6.7105e-05, 3.4281e-01, 1.2636e-02, 2.5212e-05, 1.0710e-01,
        1.4832e-04, 4.4409e-04, 1.0809e-04, 2.2008e-04],
       grad_fn=<MinBackward0>) min_indices tensor([0, 1, 2, 3, 4, 5, 0, 7, 8, 4])
i 3 sum tensor(8)
i 4 min_distances tensor([9.1698e-03, 2.6008e-04, 1.9849e-03, 5.7619e-03, 2.6151e-04, 2.3816e-02,
        3.6613e-02, 1.5

In [87]:
## check correspondence using cosine similarity
cosine_similarity = torch.nn.CosineSimilarity(dim=1)
print(signatures_full_sampling.shape)
print(signatures_down_sampled.shape)
# cosine_sim = cosine_similarity(signatures_full_sampling, signatures_down_sampled)
# print(cosine_sim)
# correspondence = torch.argmax(cosine_sim, dim=1)
# print(correspondence)

torch.Size([10, 6])
torch.Size([10, 6])


In [88]:
torch.save(signatures_full_sampling_normalized, mesh_path+ 'signatures_full_sampling_200_surfaces.pt')
torch.save(signatures_down_sampled, mesh_path+'signatures_down_sampled_200_surfaces.pt')


In [89]:
torch.__version__

'2.2.2'

In [90]:
#calculate cosine similarity between the two signatures, i want to
cosine_similarity = torch.nn.CosineSimilarity(dim=1)
cosine_similarity_mapping = []
for i in range(len(signatures_full_sampling)):
    cosine_sim = cosine_similarity(signatures_full_sampling[i], signatures_down_sampled)
    print("i", i, "cosine_sim", cosine_sim)
    correspondence = torch.argmax(torch.abs(cosine_sim))
    print("i", i, "correspondence", correspondence)
    cosine_similarity_mapping.append(correspondence)

print(cosine_similarity_mapping)
# sum all the cosine_similarity_mapping whice equal their index, for example if 0 is in 0 index, add 1 to the counter
print(torch.sum(torch.tensor(cosine_similarity_mapping) == torch.arange(len(cosine_similarity_mapping))))

i 0 cosine_sim tensor([ 1.0000,  0.7284,  0.1952, -0.1744, -0.0815, -0.1225,  0.8821, -0.4558,
        -0.8950, -0.4866], grad_fn=<SumBackward1>)
i 0 correspondence tensor(0)
i 1 cosine_sim tensor([ 0.7039,  0.9992, -0.4085,  0.3660, -0.6297, -0.0918,  0.5715, -0.8549,
        -0.7306, -0.8999], grad_fn=<SumBackward1>)
i 1 correspondence tensor(1)
i 2 cosine_sim tensor([ 0.1119, -0.2789,  0.8201, -0.9564,  0.5801,  0.7665,  0.1195,  0.2915,
         0.1070,  0.5240], grad_fn=<SumBackward1>)
i 2 correspondence tensor(3)
i 3 cosine_sim tensor([-0.1240,  0.2873, -0.8077,  0.9661, -0.6451, -0.7613, -0.0862, -0.3109,
        -0.0595, -0.5252], grad_fn=<SumBackward1>)
i 3 correspondence tensor(3)
i 4 cosine_sim tensor([-0.0878, -0.6246,  0.6824, -0.7582,  0.9993,  0.2349, -0.2066,  0.7599,
         0.0874,  0.7645], grad_fn=<SumBackward1>)
i 4 correspondence tensor(4)
i 5 cosine_sim tensor([-0.1014, -0.1425,  0.3719, -0.6592,  0.2909,  0.9934, -0.2174, -0.0574,
         0.1485,  0.2159], gra

In [91]:
signatures_full_sampling[2]

tensor([ 9.0974e+01, -6.6355e-01, -4.3205e+02, -3.2503e+03, -6.3301e+00,
        -1.9856e+03], grad_fn=<SelectBackward0>)

In [92]:
signatures_down_sampled[1]

tensor([-1.7631e-02, -2.1176e+01, -1.1015e+01,  5.3337e+00, -5.8381e+01,
         3.2079e+01], grad_fn=<SelectBackward0>)